## Load Image Tabular Data

In [2]:
import os
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import cv2
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float

pd.set_option("display.max_columns", None)
np.random.seed(42)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
import sys
#sys.path.append('..\\..\\..\\BGR')
sys.path.append('./BGR')
import bgr.soil.segmentation as bgr_seg
import bgr.soil.datasets as bgr_data

In [4]:
#path_csv = "..\\..\\..\\..\\data\\BGR_mini\\data_horizons.csv"
#path_horizons_simple = "..\\..\\..\\..\\data\\BGR_mini\\Vereinfachung_Horizontsymbole.csv"
#path_pictures = "..\\..\\..\\..\\data\\BGR_mini\\Profilbilder_no_ruler_mini_no_sky"
path_csv = "../data/BGR/data_horizons.csv"
path_horizons_simple = "../data/BGR/Vereinfachung_Horizontsymbole.csv"
path_pictures = "../data/BGR/Profilbilder_no_ruler_no_sky/"

In [5]:
df = pd.read_csv(path_csv)
print(len(df.index))
df = df.dropna(subset=['Horizontsymbol'])
print(len(df.index))
df#.head()

15207
15206


,Point,HorizontID,Obergrenze,Untergrenze,Horizontsymbol,Bodenart,Bodenfarbe,Steine,Karbonat,Humusgehaltsklasse,Durchwurzelung,Unnamed: 11,Bundesland,PointID,Unnamed: 14,profile,max_depth
0,2.0,1.0,0.0,28.0,I; Ap,Sl2,7.5YR 3/2,3.0,C0,h3,Wf4,NaN,SH,2.0,NaN,BB_0541.0,100.0
1,2.0,2.0,28.0,54.0,I; E,Sl2,10YR 3/3,3.0,C0,h2,Wf3,NaN,SH,3.0,NaN,BB_0600.0,90.0
2,2.0,3.0,54.0,68.0,II; Bv,Sl2,7.5YR 4/3,10.0,C0,h1,Wf3,NaN,SH,4.0,NaN,BB_0601.0,100.0
3,2.0,4.0,68.0,96.0,III; Bv-ilCv,fSms,7.5YR 6/4,1.0,C0,h0,Wf2,NaN,SH,5.0,NaN,BB_0659.0,100.0
4,2.0,5.0,96.0,201.0,III; ilCv,mSfs,10YR 7/4,1.0,C0,h0,NaN,NaN,SH,6.0,NaN,BB_0660.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15201,6211.0,3.0,27.0,50.0,I; Bbh+Bs,fSms,5YR 3/2,1.0,C0,h2,Wf3,NaN,NaN,NaN,NaN,NaN,NaN
15202,6211.0,4.0,50.0,70.0,I; Bbs+Bs,fSms,5YR 4/3,1.0,C0,h1,Wf2,NaN,NaN,NaN,NaN,NaN,NaN
15203,6211.0,5.0,70.0,100.0,I; Go-Bbs,fSms,7.5YR 5/4,1.0,C0,h1,Wf6,NaN,NaN,NaN,NaN,NaN,NaN
15204,6211.0,6.0,100.0,130.0,I; Go,fSms,2.5Y 6/4,1.0,C0,h0,Wf4,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_simple = pd.read_csv(path_horizons_simple)
df_simple.rename(columns={"relevanter Anteil = was sinntragend und detektierbar ist - es sind nicht alles gültige Symbole": "relevanter Anteil"}, inplace=True)
df_simple.head()

,Horiz,Anzahl im Datensatz,relevanter Anteil,stark vereinfacht
0,Ap,1762,Ap,Ap
1,Bv,626,Bv,Bv
2,ilCv,620,ilC,Cv
3,elCv,500,elC,Cv
4,Ah,455,Ah,Ah


In [7]:
# Add an extra column to the original dataset with the simplified horizon symbols

# Apply the simplification function to each row of the 'Horizontsymbol' column
col_name = "relevanter Anteil" # try to predict 'relevanter Anteil' first; otherwise, resort to 'stark vereinfacht'
df['Horizontsymbol_relevant'] = df['Horizontsymbol'].apply(lambda x: bgr_data.simplify_string(x, df_simple, col_name))

# Reorder the columns
cols = df.columns.tolist()
cols.insert(cols.index('Horizontsymbol') + 1, 'Horizontsymbol_relevant')
cols.pop() # remove the duplicate at the end
df = df[cols]
df

,Point,HorizontID,Obergrenze,Untergrenze,Horizontsymbol,Horizontsymbol_relevant,Bodenart,Bodenfarbe,Steine,Karbonat,Humusgehaltsklasse,Durchwurzelung,Unnamed: 11,Bundesland,PointID,Unnamed: 14,profile,max_depth
0,2.0,1.0,0.0,28.0,I; Ap,Ap,Sl2,7.5YR 3/2,3.0,C0,h3,Wf4,NaN,SH,2.0,NaN,BB_0541.0,100.0
1,2.0,2.0,28.0,54.0,I; E,E,Sl2,10YR 3/3,3.0,C0,h2,Wf3,NaN,SH,3.0,NaN,BB_0600.0,90.0
2,2.0,3.0,54.0,68.0,II; Bv,Bv,Sl2,7.5YR 4/3,10.0,C0,h1,Wf3,NaN,SH,4.0,NaN,BB_0601.0,100.0
3,2.0,4.0,68.0,96.0,III; Bv-ilCv,Bv-ilC,fSms,7.5YR 6/4,1.0,C0,h0,Wf2,NaN,SH,5.0,NaN,BB_0659.0,100.0
4,2.0,5.0,96.0,201.0,III; ilCv,ilC,mSfs,10YR 7/4,1.0,C0,h0,NaN,NaN,SH,6.0,NaN,BB_0660.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15201,6211.0,3.0,27.0,50.0,I; Bbh+Bs,Bbh+Bs,fSms,5YR 3/2,1.0,C0,h2,Wf3,NaN,NaN,NaN,NaN,NaN,NaN
15202,6211.0,4.0,50.0,70.0,I; Bbs+Bs,Bbs+Bs,fSms,5YR 4/3,1.0,C0,h1,Wf2,NaN,NaN,NaN,NaN,NaN,NaN
15203,6211.0,5.0,70.0,100.0,I; Go-Bbs,Go-Bbs,fSms,7.5YR 5/4,1.0,C0,h1,Wf6,NaN,NaN,NaN,NaN,NaN,NaN
15204,6211.0,6.0,100.0,130.0,I; Go,Go,fSms,2.5Y 6/4,1.0,C0,h0,Wf4,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
image_dataset = os.listdir(path_pictures)
img_files = pd.DataFrame(image_dataset, columns=['file'])
img_files['Point'] = img_files['file'].str.split("_").map(lambda x: x[1]).astype(float)
img_files.head()

,file,Point
0,BB_0896_Y_1.JPG,896.0
1,BB_0903_Y_1.JPG,903.0
2,BW_4930_Y_1.JPG,4930.0
3,BW_5183_Y_1.JPG,5183.0
4,BW_5462_Y_1.JPG,5462.0


In [9]:
df = pd.merge(df, img_files, how='inner', on='Point')
df['file'] = df['file'].map(lambda x: os.path.join(path_pictures, x))
df.head()

,Point,HorizontID,Obergrenze,Untergrenze,Horizontsymbol,Horizontsymbol_relevant,Bodenart,Bodenfarbe,Steine,Karbonat,Humusgehaltsklasse,Durchwurzelung,Unnamed: 11,Bundesland,PointID,Unnamed: 14,profile,max_depth,file
0,2.0,1.0,0.0,28.0,I; Ap,Ap,Sl2,7.5YR 3/2,3.0,C0,h3,Wf4,NaN,SH,2.0,NaN,BB_0541.0,100.0,../data/BGR/Profilbilder_no_ruler_no_sky/SH_00...
1,2.0,2.0,28.0,54.0,I; E,E,Sl2,10YR 3/3,3.0,C0,h2,Wf3,NaN,SH,3.0,NaN,BB_0600.0,90.0,../data/BGR/Profilbilder_no_ruler_no_sky/SH_00...
2,2.0,3.0,54.0,68.0,II; Bv,Bv,Sl2,7.5YR 4/3,10.0,C0,h1,Wf3,NaN,SH,4.0,NaN,BB_0601.0,100.0,../data/BGR/Profilbilder_no_ruler_no_sky/SH_00...
3,2.0,4.0,68.0,96.0,III; Bv-ilCv,Bv-ilC,fSms,7.5YR 6/4,1.0,C0,h0,Wf2,NaN,SH,5.0,NaN,BB_0659.0,100.0,../data/BGR/Profilbilder_no_ruler_no_sky/SH_00...
4,2.0,5.0,96.0,201.0,III; ilCv,ilC,mSfs,10YR 7/4,1.0,C0,h0,NaN,NaN,SH,6.0,NaN,BB_0660.0,100.0,../data/BGR/Profilbilder_no_ruler_no_sky/SH_00...


In [10]:
# Profile images only show horizons up to ~1 meter depth
df = df[df['Obergrenze'] <= 100]

# Number of horizons down to 1 meter will be used as k in KMeans per image
horizon_counts = df.groupby('file').size()
horizon_counts

file
../data/BGR/Profilbilder_no_ruler_no_sky/BB_0541_Y_1.JPG    4
../data/BGR/Profilbilder_no_ruler_no_sky/BB_0600_Y_1.JPG    6
../data/BGR/Profilbilder_no_ruler_no_sky/BB_0601_Y_1.JPG    4
../data/BGR/Profilbilder_no_ruler_no_sky/BB_0659_Y_1.JPG    4
../data/BGR/Profilbilder_no_ruler_no_sky/BB_0660_Y_1.JPG    6
                                                           ..
../data/BGR/Profilbilder_no_ruler_no_sky/TH_3489_Y_1.JPG    4
../data/BGR/Profilbilder_no_ruler_no_sky/TH_3494_Y_1.JPG    3
../data/BGR/Profilbilder_no_ruler_no_sky/TH_3496_Y_1.JPG    5
../data/BGR/Profilbilder_no_ruler_no_sky/TH_3545_Y_1.JPG    4
../data/BGR/Profilbilder_no_ruler_no_sky/TH_3596_Y_1.JPG    3
Length: 3347, dtype: int64

In [11]:
len(df)

13985

In [12]:
max(horizon_counts)

8

In [13]:
# Dictionary with lower bounds of the horizons per file
lower_bounds = {}

for index, row in df.iterrows():
    file_name = row['file']
    untergrenze = row['Untergrenze']

    if file_name not in lower_bounds:
        lower_bounds[file_name] = []

    lower_bounds[file_name].append(untergrenze)

## Segment and Group Patches

Visualize several examples as
+ original with annotated rectangular boundaries
+ SLIC patches (always about 5 times more than actual horizons)
+ final irregular horizons marked by overlapping rectangular with SLIC masks

In [13]:
max_imgs = 6

plt.figure(figsize=(3*max_imgs, 15))
for i, (file_name, k) in enumerate(horizon_counts.items()):
    if i == max_imgs: break

    lower_bounds_list = lower_bounds[file_name]
    img = cv2.imread(file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_float = img_as_float(img)#[::2, ::2])
    height, width, _ = img_float.shape

    segments_slic = slic(img_float, n_segments= 5*k, compactness=10.,
                         spacing=(2,1), sigma=10, start_label=1, convert2lab=True)
    rect_mask = bgr_seg.create_rect_mask(height, width, lower_bounds_list)
    combi_mask = bgr_seg.group_patches(lower_bounds_list, segments_slic)

    plt.subplot(3, max_imgs, 3*i+1)
    plt.imshow(mark_boundaries(img_float, rect_mask, color=(1,0,0)))
    plt.xticks([], []); plt.yticks([], [])

    plt.subplot(3, max_imgs, 3*i+2)
    plt.imshow(mark_boundaries(img_float, segments_slic, color=(1,0,0)))
    plt.xticks([], []); plt.yticks([], [])

    plt.subplot(3, max_imgs, 3*i+3)
    plt.imshow(mark_boundaries(img_float, combi_mask, color=(1,0,0)))
    plt.xticks([], []); plt.yticks([], [])
plt.tight_layout()
plt.show()

In [ ]:
max_imgs = 100

for i, (file_name, k) in enumerate(horizon_counts.items()):
    if i%10==0: print(i)
    if i == max_imgs: break

    lower_bounds_list = lower_bounds[file_name]
    img = cv2.imread(file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_float = img_as_float(img)#[::2, ::2]) # image is in [0, 1]
    height, width, _ = img_float.shape

    segments_slic = slic(img_float, n_segments= 5*k, compactness=10.,
                         spacing=(2,1), sigma=10, start_label=1, convert2lab=True)
    rect_mask = bgr_seg.create_rect_mask(height, width, lower_bounds_list)
    combi_mask = bgr_seg.group_patches(lower_bounds_list, segments_slic)

    img_rect_mask = mark_boundaries(img_float, rect_mask, color=(0,0,1), outline_color=(0,0,1), mode='thick')
    img_horizon = mark_boundaries(img_float, combi_mask, color=(0,0,1), outline_color=(0,0,1), mode='thick')

    # Convert to uint8 for saving
    img_rect_mask = cv2.cvtColor((img_rect_mask * 255).astype('uint8'), cv2.COLOR_BGR2RGB)
    img_horizon   = cv2.cvtColor((img_horizon * 255).astype('uint8'), cv2.COLOR_BGR2RGB)

    # Combine the images horizontally
    pad = np.full((img.shape[0], 50, 3), (255, 255, 255), dtype=img.dtype) # padding as a blank image
    combined_img = np.hstack((img_rect_mask, pad, img_horizon))

    cv2.imwrite('..\\..\\..\\..\\data\\BGR_mini\\Beispiele_SLIC_Group\\rect_and_horizons\\' + file_name.split('\\')[-1], combined_img)

Store extra dataset of segmented horizons

In [ ]:
from joblib import Parallel, delayed

horizon_dir = "../data/BGR/Horizonte/"
def process_file(file_name, k, lower_bounds, horizon_dir):
    try:
        lower_bounds_list = lower_bounds[file_name]
        img = cv2.imread(file_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_float = img_as_float(img)
        height, width, _ = img_float.shape

        segments_slic = slic(img_float, n_segments=5 * k, compactness=10.,
                             spacing=(2, 1), sigma=10, start_label=1, convert2lab=True)
        combi_mask = bgr_seg.group_patches(lower_bounds_list, segments_slic)

        bgr_seg.save_horizon_overlays(combi_mask, img, horizon_dir, file_name)
    except Exception as e:
        print(f"Error processing file: {file_name}")
        print(f"Exception: {e}")

# Run in parallel
Parallel(n_jobs=16)(delayed(process_file)(file_name, k, lower_bounds, horizon_dir) for file_name, k in horizon_counts.items())

**Note:** some samples stop with the lowest boundary at less than -1M e.g. ID-4554. For these samples, there were no horizon images stored separately.